In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

# Load the data
data = pd.read_csv('META.csv')

# Selecting relevant features
features = data[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]

# Normalize the features
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

# Create sequences
SEQ_LENGTH = 100

def create_sequences(data, seq_length):
    xs = []
    ys = []
    for i in range(len(data) - seq_length):
        x = data[i:i+seq_length]
        y = data[i+seq_length, 0]  # Predicting the 'Open' price
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

X, y = create_sequences(scaled_features, SEQ_LENGTH)

# Split the data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [4]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(units=100, activation='tanh', return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dropout(0.3))
model.add(LSTM(units=100, activation='tanh', return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(units=100, activation='tanh'))
model.add(Dropout(0.3))
model.add(Dense(units=50, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
history = model.fit(x_train, y_train, epochs=100, batch_size=64, validation_split=0.2, callbacks=[
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),
    tf.keras.callbacks.ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
])



Epoch 1/100


11/11 [==============================] - 15s 619ms/step - loss: 0.0464 - mae: 0.1629 - val_loss: 0.0025 - val_mae: 0.0407
Epoch 2/100


C:\Users\Hamed\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


11/11 [==============================] - 5s 419ms/step - loss: 0.0139 - mae: 0.0918 - val_loss: 0.0014 - val_mae: 0.0300
Epoch 3/100
11/11 [==============================] - 4s 405ms/step - loss: 0.0117 - mae: 0.0837 - val_loss: 0.0041 - val_mae: 0.0586
Epoch 4/100
11/11 [==============================] - 4s 405ms/step - loss: 0.0095 - mae: 0.0731 - val_loss: 0.0031 - val_mae: 0.0502
Epoch 5/100
11/11 [==============================] - 4s 394ms/step - loss: 0.0088 - mae: 0.0698 - val_loss: 0.0016 - val_mae: 0.0347
Epoch 6/100
11/11 [==============================] - 4s 399ms/step - loss: 0.0075 - mae: 0.0650 - val_loss: 8.8056e-04 - val_mae: 0.0218
Epoch 7/100
11/11 [==============================] - 4s 401ms/step - loss: 0.0079 - mae: 0.0670 - val_loss: 7.8791e-04 - val_mae: 0.0194
Epoch 8/100
11/11 [==============================] - 4s 390ms/step - loss: 0.0076 - mae: 0.0647 - val_loss: 0.0010 - val_mae: 0.0228
Epoch 9/100
11/11 [==============================] - 4s 397ms/step - loss

In [5]:
# Prepare the last sequence for prediction (assuming you have up-to-date data)
last_sequence = scaled_features[-SEQ_LENGTH:]
last_sequence = np.expand_dims(last_sequence, axis=0)

# Load the best model and make predictions
model.load_weights('best_model.h5')
predicted_open_price = model.predict(last_sequence)

# Inverse transform the prediction to get the actual price
predicted_open_price = scaler.inverse_transform(np.concatenate([predicted_open_price, np.zeros((predicted_open_price.shape[0], scaled_features.shape[1] - 1))], axis=1))[:, 0]

print(f"Predicted Open Price for July 17, 2024: {predicted_open_price[0]}")

1/1 [==============================] - 1s 1s/step
Predicted Open Price for July 17, 2024: 456.3116942739923
